In [1]:
from langdetect import detect
import pandas as pd
from sklearn.metrics import accuracy_score
import numpy as np
import os

In [2]:
df = pd.read_csv("../../data/qald-9-test.csv")

In [3]:
df= df.dropna(subset=['questions'])

In [4]:
df_en = df[df["lang"]=='en']
df_fr = df[df["lang"]=='fr']
df_de = df[df["lang"]=='de']

In [5]:
def extractLan(f,n):
    result=[]
    count=0
    while count < n:
        line = f.readline()
        if line == "\n":
            continue
        result.append(line[:3])
        count+=1
    return result


def run_opennlp(data):
    with open('opennlp_input.txt', 'w') as f:
        for q in data["questions"]:
            f.write(q+"\n\n")
    os.system("/home/oshara/GSoC/DBPedia/openNLP/apache-opennlp-1.9.2/bin/opennlp LanguageDetector /home/oshara/GSoC/DBPedia/openNLP/langdetect-183.bin </home/oshara/GSoC/DBPedia/QLang/evaluation/Eval_opennlp/opennlp_input.txt> /home/oshara/GSoC/DBPedia/QLang/evaluation/Eval_opennlp/opennlp_output.txt")
    n=len(data)
    with open('opennlp_output.txt', 'r') as f:
        return extractLan(f,n)
        

In [6]:
re_en = run_opennlp(df_en)
re_fr = run_opennlp(df_fr)
re_de = run_opennlp(df_de)

In [7]:
# English, Spanish, German, Italian, French, Dutch, Romanian
# {'en': 0, 'de': 1, 'es': 2, 'it': 3, 'fr': 4, 'nl': 5, 'ro': 6}
# label = {'eng': 0, "spa": 2, "deu": 1, "ita": 3, "fra": 4, "nld": 5, "ron": 6, "other": 7}
label = {'eng': 0, "deu": 1, "fra": 2,"other": 3}

In [8]:
def replace_label(result):
    for i in range(len(result)):
        key = result[i]
        if key in label:
            result[i] = label[key]
        else:
            result[i] = 7

replace_label(re_en)
replace_label(re_de)
replace_label(re_fr)

In [9]:
possible_labels = df["lang"].unique()

label_dict = {"en":0,"de":1,"fr":2}


df_en['lang'] = df_en.lang.replace(label_dict)
df_de['lang'] = df_de.lang.replace(label_dict)
df_fr['lang'] = df_fr.lang.replace(label_dict)

/home/oshara/Programs/anaconda2/envs/jupy/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/oshara/Programs/anaconda2/envs/jupy/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/oshara/Programs/anaconda2/envs/jupy/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [10]:
# languages
# q3 ['en', 'de', 'es', 'it', 'fr', 'nl']
# q4 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q5 ['en', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q6 ['en', 'fa', 'de', 'es', 'it', 'fr', 'nl', 'ro']
# q7 ['en', 'de', 'it', 'fr']
# q8 ['en']

In [11]:
# EN
# q3 0.9797979797979798
# q4 0.92
# q5 0.91
# q6 0.94
# q7 0.96
# q8 0.95

In [12]:
# FR
# q3 0.8686868686868687
# q4 0.90
# q5 0.8
# q6 0.82
# q7 0.90
# q8 

In [13]:
# DE
# q3 0.8888888888888888
# q4 0.80
# q5 0.81
# q6 0.74
# q7 0.79
# q8 

In [14]:
accuracy_score(df_en["lang"], re_en)

0.9333333333333333

In [15]:
accuracy_score(df_fr["lang"], re_fr)

0.8933333333333333

In [16]:
accuracy_score(df_de["lang"], re_de)

0.82